In [1]:
import sys
sys.path.append("..")  # or wherever your src/ lives relative to the notebook


In [27]:
from music21 import *
import torch.nn as nn
import torch.optim as optim

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch

from src.models import SeqPredictorLSTM
from src.utils import ToySequenceDataset
from src.utils import sort_and_batch, custom_collate_fn


In [ ]:
# Example vocab
vocab = {
    "<PAD>": 0,
    "<SOS>": 1,
    "<EOS>": 2,
    "the": 3,
    "cat": 4,
    "sat": 5,
    "on": 6,
    "mat": 7,
    "dog": 8,
    "barked": 9,
    "loudly": 10,
    "softly": 11,
    "quickly": 12,
    "slowly": 13,
}
# Example indices
rev_vocab = {v: k for k, v in vocab.items()}
pad_idx = vocab["<PAD>"]
sos_idx = vocab["<SOS>"]
eos_idx = vocab["<EOS>"]

toy_sequences = [
    [3, 4, 5, 6, 7],
    [3, 8, 9, 10],
    [4],
    [3, 8, 5, 6, 3, 4],
]


In [26]:
dataset = ToySequenceDataset(toy_sequences, sos_idx, eos_idx)

batches = sort_and_batch(dataset, batch_size=2)

dataloader = DataLoader(
    dataset,
    batch_sampler=batches,
    collate_fn=custom_collate_fn(pad_idx)
)

# Test
for batch in dataloader:
    print(batch)

Sample 0: (tensor([1, 3, 4, 5, 6, 7]), tensor([3, 4, 5, 6, 7, 2]))
Length: 6
Value: <SOS> the cat sat on mat
Label: the cat sat on mat <EOS>

Sample 1: (tensor([ 1,  3,  8,  9, 10]), tensor([ 3,  8,  9, 10,  2]))
Length: 5
Value: <SOS> the dog barked loudly
Label: the dog barked loudly <EOS>

Sample 2: (tensor([1, 4]), tensor([4, 2]))
Length: 2
Value: <SOS> cat
Label: cat <EOS>

Sample 3: (tensor([1, 3, 8, 5, 6, 3, 4]), tensor([3, 8, 5, 6, 3, 4, 2]))
Length: 7
Value: <SOS> the dog sat on the cat
Label: the dog sat on the cat <EOS>



In [30]:
model = SeqPredictorLSTM(vocab_size=len(vocab), pad_idx=pad_idx)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
epochs = 10

model.fit(dataloader=dataloader, epochs=epochs, optimizer=optimizer, criterion=loss_fn, device="mps")

Epoch 1/10


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [5]:
def predict_next_tokens(model, input_seq, vocab, max_len=10):
    model.eval()
    idx_to_word = {i: w for w, i in vocab.items()}
    
    input_tensor = torch.tensor([input_seq], dtype=torch.long)  # batch of 1
    with torch.no_grad():
        output = model(input_tensor)  # (1, seq_len, vocab_size)
        predictions = output.argmax(-1).squeeze(0).tolist()
    
    return [idx_to_word[i] for i in predictions]

In [ ]:
input_seq = [sos_idx, vocab["the"]]  # → "<SOS> the cat"
print(predict_next_tokens(model, input_seq, vocab))

In [23]:
input_tensor = torch.tensor([input_seq], dtype=torch.long)  # batch of 1

output = model(input_tensor)

In [ ]:
predicted_probs = torch.softmax(output, dim=-1)
predicted = predicted_probs.argmax(dim=-1)


In [36]:
input_tensor
target_tensor = torch.tensor([[3, 4]], dtype=torch.long)  # batch of 1

In [ ]:
predicted_probs

In [37]:
a = (predicted == target_tensor).sum().item()

In [ ]:
a
